In [1]:
#ライブラリ
import tweepy
import nltk
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [40]:
#Twitter API config読み込み
config = pd.read_csv("./config.csv")

In [47]:
# Twitter API config
twitterApiKey = config['twitterApiKey'][0]
twitterApiSecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]

In [54]:
# Authenticate
auth = tweepy.OAuthHandler(twitterApiKey, twitterApiSecret)
auth.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [55]:
# キーワードからツイートを取得
s='btc min_faves:1000 min_retweets:1000 exclude:retweets'
tweets = tweepy.Cursor(api.search,  
                        count=10000,
                        q=s,
                        exclude_replies=True,
                        tweet_mode='extended',
                        show_user=True,
                        lang = 'en',
                        since='2021-07-08',
                        until='2021-07-13',
                        ).items(1000000);

In [ ]:
#ツイートのリスト
t_data = []
for tweet in tweets:
    t_data.append([tweet.user.name,'@'+tweet.user.screen_name,tweet.user.followers_count,tweet.full_text.replace('\n',''),tweet.favorite_count,tweet.retweet_count])

In [56]:
#処理前ツイート出力
df = pd.DataFrame(data=t_data, columns=['Name','User Name','Followers','Tweet','Favorite','RT'])
df.head(10)

In [57]:
#処理前ツイート先頭10件
df.head(10)

,Tweet
0,The Blue Dollar (Informal ARS/USD exchange rat...
1,BTCEarn Amazing Rewards Coin!💎\n\nNew Presale ...
2,📣$70K #BTC+#BSC GIVEAWAY 📣\n\nOn behalf of the...
3,FINALLY PHASE 1 HAS BEEN LAUNCHED!!!!!!!!!!!!!...
4,"I am here to unite Paraguay 🇵🇾, that is why we..."
5,$1000 #BNB &amp; NFT GIVEAWAY (via @Treasurela...
6,🐕 DogeBTC 🐕 Whitelist is now live! Join here: ...
7,Apple co-founder Steve Wozniak Calls #Bitcoin ...
8,@MattWallace888 BTC &amp; ETH are pursuing a m...
9,"💎$15,000 GIVEAWAY💎\nI will send 50 #bnb to 5 p..."


In [52]:
#処理前ツイート末尾10件
df.tail(10)

,Tweet
0,The Blue Dollar (Informal ARS/USD exchange rat...
1,BTCEarn Amazing Rewards Coin!💎\n\nNew Presale ...
2,📣$70K #BTC+#BSC GIVEAWAY 📣\n\nOn behalf of the...
3,FINALLY PHASE 1 HAS BEEN LAUNCHED!!!!!!!!!!!!!...
4,"I am here to unite Paraguay 🇵🇾, that is why we..."
5,$1000 #BNB &amp; NFT GIVEAWAY (via @Treasurela...
6,🐕 DogeBTC 🐕 Whitelist is now live! Join here: ...
7,Apple co-founder Steve Wozniak Calls #Bitcoin ...
8,@MattWallace888 BTC &amp; ETH are pursuing a m...
9,"💎$15,000 GIVEAWAY💎\nI will send 50 #bnb to 5 p..."


In [53]:
#処理前出力
df.to_csv("main.csv")

In [ ]:
#ツイート前処理
def cleanUpTweet(txt):
    # Remove mentions
    txt = re.sub(r'@[A-Za-z0-9_]+', '', txt)
    # Remove hashtags
    txt = re.sub(r'#', '', txt)
    # Remove urls
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt)
    return txt

In [ ]:
#ステミング
from nltk.stem.porter import PorterStemmer as PS
ps = PS()

In [ ]:
#VADER感情分析器インポート
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader_analyzer = SentimentIntensityAnalyzer()

In [ ]:
#感情出力関数
def getTextPolarity(txt):
    return vader_analyzer.polarity_scores(txt)
def getTextPos(txt):
    return txt.get("pos")
def getTextNeu(txt):
    return txt.get("neu")
def getTextNeg(txt):
    return txt.get("neg")
def getTextCom(txt):
    return txt.get("compound")

def getTextAnalysis(l):
    s = l.get("compound")
    if s < 0:
        return "Negative"
    elif s == 0:
        return "Neutral"
    else:
        return "Positive"

In [ ]:
#表に分析結果を追加
df['Score'] = df['Tweet'].apply(getTextPolarity)
df['neg'] = df['Score'].apply(getTextNeg)
df['neu'] = df['Score'].apply(getTextNeu)
df['pos'] = df['Score'].apply(getTextPos)
df['compound'] = df['Score'].apply(getTextCom)
df['Sentiment'] = df['Score'].apply(getTextAnalysis)

In [ ]:
#感情数値表示
df.drop(columns='Score')

In [ ]:
#結果を表に上書き
df.to_csv("main.csv")